In [2]:
import pandas as pd 
import cv2 
import numpy as np 


In [3]:
df = pd.read_csv('train.csv')

In [4]:
df.head()

,0,1
0,data/HockeyFights/no/no340_xvid.avi,0
1,data/HockeyFights/yes/fi166_xvid.avi,1
2,data/HockeyFights/no/no220_xvid.avi,0
3,data/HockeyFights/no/no365_xvid.avi,0
4,data/HockeyFights/yes/fi249_xvid.avi,1


In [5]:
x = df.iloc[:,0].values
y = df.iloc[:,1].values

In [7]:
for i in x:
    cap = cv2.VideoCapture(i)
    frame_count=0
    previous_frame = 0
    o_mag=0
    o_ang=0
    
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            if frame_count>0:
                cv2.imshow('frame',frame)
                gray1 = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                gray2 = cv2.cvtColor(previous_frame, cv2.COLOR_BGR2GRAY)
                gray1 = gray1.reshape(gray2.shape)
                flow = cv2.calcOpticalFlowFarneback(gray2,gray1, None, 0.5, 3, 15, 3, 5, 1.2, 0)
                mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
                magnitude = abs(mag - o_mag)
                angle = abs(ang - o_ang)
                o_ang = ang 
                o_mag = mag
#                 print(magnitude.shape)
#                 print(angle.shape)
                magnitude = cv2.medianBlur(magnitude,5).astype('uint8')
                angle = cv2.medianBlur(angle,5).astype('uint8')
                Mag_adaptive = cv2.adaptiveThreshold(magnitude,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
                Ang_adaptive = cv2.adaptiveThreshold(angle,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
                
                ret ,Mag_otsu = cv2.threshold(magnitude,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
                ret, Ang_otsu = cv2.threshold(angle,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
                ret,Mag_mean = cv2.threshold(magnitude,127,255,cv2.THRESH_BINARY)
                ret,Ang_mean = cv2.threshold(angle,127,255,cv2.THRESH_BINARY)
               
                
                hsv = np.zeros_like(previous_frame)
                hsv[...,1] = 255
                hsv[...,0] = ang*180/np.pi/2
                hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
                rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
                
                cv2.imshow('frame1',rgb)
                hsv = np.zeros_like(previous_frame)
                hsv[...,1] = 255
                hsv[...,0] = angle*180/np.pi/2
                hsv[...,2] = cv2.normalize(magnitude,None,0,255,cv2.NORM_MINMAX)
                rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
                
                cv2.imshow('frame2',rgb)
                
                cv2.imshow('Mag_adaptive',Mag_adaptive)
                cv2.imshow('Ang_adaptive',Ang_adaptive)
                cv2.imshow('Mag_otsu',Mag_otsu)
                cv2.imshow('Ang_otsu',Ang_otsu)
                cv2.imshow('Mag_mean',Mag_mean)
                cv2.imshow('Ang_mean',Ang_mean)
                
                
                
                

                hsv = np.zeros_like(previous_frame)
                hsv[...,1] = 255
                hsv[...,0] = angle*180/np.pi/2
                hsv[...,2] = cv2.normalize(magnitude,None,0,255,cv2.NORM_MINMAX)
                rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
                
                cv2.imshow('frame2',rgb)
            
#             cv2.imshow('frame',gray)
            previous_frame = frame 
            frame_count = frame_count+1
            
            if cv2.waitKey(30) & 0xFF == ord('q'):
                break

        else:
            break

    cap.release()
    #out.release()
    cv2.destroyAllWindows() 

KeyboardInterrupt: 